In [1]:
import json
f = open("./data/data/movie3.json", "r")
movie_data = json.load(f)
f.close()

# project_budget nan값 갯수 확인하기

In [36]:
not_korean_list =[]
for i in range(932):
    if not any(u"한국" in s for s in movie_data[i]['nation']):
         not_korean_list.append(i)

print len(not_korean_list) 

872


In [38]:
count = 0
for i in not_korean_list:
    if movie_data[i]['project_budget'] != 'NA':

        count +=1
print count

622


In [39]:
nan_budget_list = []
for i in not_korean_list:
    if movie_data[i]['project_budget'] == 'NA':
        nan_budget_list.append(movie_data[i]['Mname'])
        
print len(nan_budget_list)

250


# project_budget data 채우기

In [9]:
import pandas as pd
kaggle_data = pd.read_csv('./data/movie_metadata.csv')

In [10]:
kaggle_data.tail()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,...,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
5038,Color,Scott Smith,1.0,87.0,2.0,318.0,Daphne Zuniga,637.0,NaN,Comedy|Drama,...,6.0,English,Canada,NaN,NaN,2013.0,470.0,7.7,NaN,84
5039,Color,NaN,43.0,43.0,NaN,319.0,Valorie Curry,841.0,NaN,Crime|Drama|Mystery|Thriller,...,359.0,English,USA,TV-14,NaN,NaN,593.0,7.5,16.00,32000
5040,Color,Benjamin Roberds,13.0,76.0,0.0,0.0,Maxwell Moody,0.0,NaN,Drama|Horror|Thriller,...,3.0,English,USA,NaN,1400.0,2013.0,0.0,6.3,NaN,16
5041,Color,Daniel Hsia,14.0,100.0,0.0,489.0,Daniel Henney,946.0,10443.0,Comedy|Drama|Romance,...,9.0,English,USA,PG-13,NaN,2012.0,719.0,6.3,2.35,660
5042,Color,Jon Gunn,43.0,90.0,16.0,16.0,Brian Herzlinger,86.0,85222.0,Documentary,...,84.0,English,USA,PG,1100.0,2004.0,23.0,6.6,1.85,456


In [11]:
import re
from collections import OrderedDict

k_budget_dict = OrderedDict()
for i in range(len(kaggle_data['budget'])):
    movie_title = re.sub('\xc2\xa0','',kaggle_data['movie_title'][i])
    k_budget_dict[movie_title] = kaggle_data['budget'][i]

In [13]:
m_name = k_budget_dict.keys()

In [30]:
count =0
for i in range(len(nan_budget_list)):
    if any(nan_budget_list[i].encode('utf-8') in s for s in m_name):
        count +=1
print count

116


In [44]:
import math

In [ ]:
count =0
update_dict = {}
for i in range(len(nan_budget_list)):
    if any(nan_budget_list[i].encode('utf-8') in s for s in m_name):
        try:
            if not math.isnan(k_budget_dict[nan_budget_list[i]]):
                update_dict[nan_budget_list[i]] = k_budget_dict[nan_budget_list[i]]
                count +=1
        except:
            pass
print count
print update_dict

# db에 update

In [17]:
from pymongo import MongoClient

In [20]:
mongo = MongoClient(mongo_server, 27017)

In [21]:
movie = mongo.project.movie

In [52]:
m_names = update_dict.keys()

In [59]:
for i in range(len(update_dict)):
    result = movie.update_one({'Mname': m_names[i]}, {'$set' :{'project_budget':update_dict[m_names[i]]}} )
print result.modified_count

1


# actor, director nan개수 확인하기

In [ ]:
for i in not_korean_list:
    print movie_data[i]['Mname']
    print movie_data[i]['actor']
    print

In [442]:
count = 0
nan_actor_list = []
for i in not_korean_list:
    try:
        if movie_data[i]['actor']['People'] == u'/people/?ref=ft':
            nan_actor_list.append(movie_data[i]['Mname'])
            count += 1
    except:
        pass
print count
print len(not_korean_list)
print len(nan_actor_list)

164
872
164


In [ ]:
for i in not_korean_list:
    print movie_data[i]['Mname']
    print movie_data[i]['director']
    print

In [41]:
count = 0
nan_director_list = []
for i in not_korean_list:
    try:
        if movie_data[i]['director']['People'] == u'/people/?ref=ft':
            nan_director_list.append(movie_data[i]['Mname'])
            count += 1
    except:
        pass
print count
print len(not_korean_list)
print len(nan_director_list)

189
872
189


# director nan값 채우기

In [ ]:
for i in not_korean_list:
    print movie_data[i]['director']

In [ ]:
k_director_dict = {}
for i in range(len(kaggle_data['director_name'])):
    movie_title = re.sub('\xc2\xa0','',kaggle_data['movie_title'][i])
    k_director_dict[movie_title] = kaggle_data['director_name'][i]
print k_director_dict

In [466]:
m_name = k_director_dict.keys()

In [467]:
count = 0
update_dict = {}
for i in range(len(nan_director_list)):
    if any(nan_director_list[i].encode('utf-8') in s for s in m_name):
        try:
            k_director_dict[nan_director_list[i]]
            #print nan_director_list[i], k_director_dict[nan_director_list[i]]
            update_dict[nan_director_list[i]] = k_director_dict[nan_director_list[i]]
            count += 1
        except:
            pass
print count
print len(update_dict)

85
85


In [468]:
movie = mongo.project.movie

In [469]:
m_names = update_dict.keys()

In [473]:
for i in range(len(update_dict)):
    result = movie.update_one({'Mname': m_names[i]}, {'$set' :{'director':update_dict[m_names[i]]}} )
print result.modified_count

1


# director 제작영화 list 만들기

In [474]:
directors = k_director_dict.values()

In [475]:
d_names = update_dict.values()

In [ ]:
from collections import Counter
directors_dict = Counter(directors)

for d in d_names:
    print d, directors_dict[d]


In [482]:
all_movie_name = k_director_dict.keys()

In [483]:
director_movies_dict = {}
for i in range(len(update_dict)):
    movie_list = []
    for j in range(len(k_director_dict)):
        if update_dict[m_names[i]] == k_director_dict[all_movie_name[j]]:
            movie_list.append(all_movie_name[j])
    director_movies_dict[update_dict[m_names[i]]] = movie_list

In [485]:
director = mongo.project.director

In [ ]:
for i in range(len(director_movies_dict)):
    value_dict = {}
    for j in range(len(director_movies_dict.values()[i])):
        value_dict[director_movies_dict.values()[i][j]] = 'NA'

    director.insert({'movie': value_dict, 'name': director_movies_dict.keys()[i]}, check_keys=False)

# director 제작영화 매출액 구하기

In [ ]:
count = 0
movie_gross_list = []

for i in range(len(director_movies_dict)):
    director_gross_dict = {}
    for j in range(len(director_movies_dict.values()[i])):
        for k in not_korean_list:
            if director_movies_dict.values()[i][j] == movie_data[k]['Mname']:
                #print movie_list[i], movie_data[j]['gross_kor']
                
                director_gross_dict[re.sub(r'\.','',director_movies_dict.values()[i][j])] = movie_data[k]['gross_kor']
                count+= 1
    print director_movies_dict.keys()[i], director_gross_dict
    movie_gross_list.append(director_gross_dict)

print count
print len(movie_gross_list)

# db에 저장

In [488]:
director = mongo.project.director

In [499]:
for i in range(len(movie_gross_list)):
    for j in range(len(movie_gross_list[i])):
        gross_list = ['NA','NA','NA',movie_gross_list[i].values()[j]]
        result = director.update_one({'name': director_movies_dict.keys()[i]}, {'$set': {'movie.{}'.format(movie_gross_list[i].keys()[j]): gross_list}})

# actor nan값 채우기

In [ ]:
for i in not_korean_list:
    print movie_data[i]['actor']

In [ ]:
k_actor_dict = {}
for i in range(len(kaggle_data['actor_1_name'])):
    actor_list = []
    movie_title = re.sub('\xc2\xa0','',kaggle_data['movie_title'][i])
    actor_list.append(kaggle_data['actor_1_name'][i])
    actor_list.append(kaggle_data['actor_2_name'][i])
    actor_list.append(kaggle_data['actor_3_name'][i])                 
    k_actor_dict[movie_title] = actor_list
print k_actor_dict

In [445]:
print len(k_actor_dict)

4917


In [447]:
count = 0
update_dict = {}
for i in range(len(nan_actor_list)):
    if any(nan_actor_list[i].encode('utf-8') in s for s in k_actor_dict.keys()):
        try:
            k_actor_dict[nan_actor_list[i]]
            #print nan_actor_list[i], k_actor_dict[nan_actor_list[i]]
            update_dict[nan_actor_list[i]] = k_actor_dict[nan_actor_list[i]]
            count += 1
        except:
            pass
print count
print len(update_dict)

37
37


In [461]:
for i in range(len(update_dict)):
    for j in range(3):
        actor = re.sub('\.','', update_dict.values()[i][j])
        result = movie.update_one({'Mname': update_dict.keys()[i]}, {'$set' :{'actor.{}'.format(actor): 'NA'}}, upsert= True )
    

# actor 출연영화 list 만들기

In [ ]:
update_dict.values()

In [ ]:
actor_movies_dict = {}
for i in range(len(update_dict)):
    movie_list = []
    for j in range(len(k_actor_dict)):
        if update_dict[m_names[i]] == k_director_dict[all_movie_name[j]]:
            movie_list.append(all_movie_name[j])
    director_movies_dict[update_dict[m_names[i]]] = movie_list